In [1]:
import pandas as pd
import numpy as np

In [86]:
all_transactions = pd.read_csv('all_transactions.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
transactions['value'] = transactions['value'].astype('float64')

In [4]:
all_transactions.columns

Index(['from', 'to', 'value', 'time', 'block'], dtype='object')

In [5]:
trans_genesis = all_transactions[all_transactions['block']==0]

In [7]:
trans_genesis.head()

,from,to,value,time,block
0,ethereum,0000000000000000000000000000000000000000,5.000000e+18,0,0
204952137,genesis,756f45e3fa69347a9a973a725e3c98bc4db0b5a0,2.000000e+20,0,0
204952138,genesis,f42f905231c770f0a406f2b768877fb49eee0f21,1.970000e+20,0,0
204952139,genesis,2489ac126934d4d6a94df08743da7b7691e9798e,1.000000e+21,0,0
204952140,genesis,ddf5810a0eb2fb2e32323bb2c99509ab320f24ac,1.790000e+22,0,0


In [8]:
val = trans_genesis['value'].sum()

In [9]:
from web3 import Web3

In [10]:
Web3.fromWei(val, 'ether')

Decimal('72009995.499480001716158464')

In [29]:
# a dictionary mapping address to the corresponding genesis level
from multiprocessing import Process,Manager
manager = Manager()
genesisLevel = manager.dict()

In [18]:
curr_level = 0
filtered_trans = all_transactions[(all_transactions['from'] == 'ethereum') | (all_transactions['from'] == 'genesis')]

In [20]:
filtered_trans['from'].unique()

array(['ethereum', 'genesis'], dtype=object)

In [21]:
# gather all addresses
addresses = filtered_trans['to'].unique()

In [23]:
len(addresses)

13044

In [24]:
reward_transaction = all_transactions[all_transactions['from']=='ethereum']

In [25]:
reward_transaction.count()

from     6458881
to       6458881
value    6458881
time     6458881
block    6458881
dtype: int64

In [28]:
len(reward_transaction['to'].unique())

4176

In [32]:
x = [1,2,3,4,5,6,7,8,9]
print(x[0:3])
print(x[3:6])
print(x[6:])


[1, 2, 3]
[4, 5, 6]
[7, 8, 9]


In [33]:
curr_level = 1

In [37]:
partition_size = int(len(addresses)/3) + 1
start_i = 0
end_i = start_i + partition_size
for i in range(3):
    if i==2:
        print(len(addresses[start_i:]))
    else:
        print(len(addresses[start_i:end_i]))
    start_i = end_i
    end_i = start_i + partition_size
print(len(addresses))    

4349
4349
4346
13044


In [24]:
# fill dictionary
def fillLevels(address_list, genesisLevel, curr_level):
    for addr in address_list:
        genesisLevel[addr] = curr_level

In [40]:
len(genesisLevel)

13044

In [47]:
filtered_trans = all_transactions[all_transactions['from'].isin(addresses) ]['to']

In [49]:
len(filtered_trans.unique())

990626

In [52]:
len(addresses)

13044

In [84]:
from multiprocessing import Process,Manager
manager = Manager()
genesisLevel = manager.dict()
curr_level = 0

In [73]:
filtered_trans = transactions[(transactions['from'] == 'ethereum') | (transactions['from'] == 'genesis')]['to']
addresses = filtered_trans.unique()

In [13]:
new_transactions = all_transactions[~all_transactions['to'].isin(addresses)]

In [69]:
transactions.count()

from     204961030
to       204961030
value    204961030
time     204961030
block    204961030
dtype: int64

In [15]:
new_transactions.count()

from     198182969
to       198182969
value    198182969
time     198182969
block    198182969
dtype: int64

In [74]:
while len(addresses)>0:
    curr_level+=1
    if curr_level >= 5000:
        break
    print('looking into level: ',curr_level)
    jobs = []
    if len(addresses)>100:
        job_cnt = 4
    else:
        job_cnt = 1
    partition_size = int(len(addresses)/job_cnt) + 1
    start_i = 0
    end_i = start_i + partition_size
    for i in range(job_cnt):
        if i==job_cnt-1:
            p = Process(target=fillLevels, args=(addresses[start_i:],genesisLevel,curr_level))
        else:
            p = Process(target=fillLevels, args=(addresses[start_i:end_i],genesisLevel,curr_level))
        start_i = end_i
        end_i = start_i + partition_size
        jobs.append(p)
        p.start()

    for proc in jobs:
        proc.join()
    
    print('filled the dictionary level values')
    transactions = transactions[~transactions['to'].isin(addresses)]
    filtered_trans = transactions[transactions['from'].isin(addresses)]['to']
    addresses = filtered_trans.unique()
    #addresses = np.setdiff1d(new_addresses,addresses) # remove addresses of higher level
    print('obtained addresses for the next level')
    print('addresses obtained: ',len(addresses))

looking into level:  1
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  988426
looking into level:  2
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  4708611
looking into level:  3
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  8533293
looking into level:  4
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  3331247
looking into level:  5
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1697040
looking into level:  6
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1635002
looking into level:  7
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1917090
looking into level:  8
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1949139
l

filled the dictionary level values
obtained addresses for the next level
addresses obtained:  228
looking into level:  69
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  226
looking into level:  70
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  220
looking into level:  71
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  217
looking into level:  72
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  209
looking into level:  73
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  203
looking into level:  74
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  198
looking into level:  75
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  200
looking into level:  76
filled the dictionary le

obtained addresses for the next level
addresses obtained:  64
looking into level:  136
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  62
looking into level:  137
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  60
looking into level:  138
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  59
looking into level:  139
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  59
looking into level:  140
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  59
looking into level:  141
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  58
looking into level:  142
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  58
looking into level:  143
filled the dictionary level values
obtained addresses for t

filled the dictionary level values
obtained addresses for the next level
addresses obtained:  25
looking into level:  204
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  25
looking into level:  205
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  25
looking into level:  206
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  25
looking into level:  207
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  24
looking into level:  208
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  24
looking into level:  209
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  24
looking into level:  210
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  24
looking into level:  211
filled the dictionary le

obtained addresses for the next level
addresses obtained:  15
looking into level:  271
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  15
looking into level:  272
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  15
looking into level:  273
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  15
looking into level:  274
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  15
looking into level:  275
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  15
looking into level:  276
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  15
looking into level:  277
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  15
looking into level:  278
filled the dictionary level values
obtained addresses for t

...

obtained addresses for the next level
addresses obtained:  2
looking into level:  4730
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4731
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4732
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4733
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4734
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4735
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4736
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4737
filled the dictionary level values
obtained addresses for t

filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4798
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4799
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4800
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4801
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4802
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4803
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4804
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4805
filled the dictionary le

obtained addresses for the next level
addresses obtained:  2
looking into level:  4865
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4866
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4867
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4868
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4869
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4870
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4871
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4872
filled the dictionary level values
obtained addresses for t

filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4933
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4934
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4935
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4936
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4937
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4938
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4939
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4940
filled the dictionary le

obtained addresses for the next level
addresses obtained:  2


In [75]:
transactions.count()

from     273147
to       273147
value    273147
time     273147
block    273147
dtype: int64

In [81]:
with open('genesisLevel.csv','w') as ft:
    ft.write('address,level')
    ft.write('\n')
    for k in genesisLevel.keys():
        ft.write(k+','+str(genesisLevel[k]))
        ft.write('\n')

In [85]:
genesisLevel.keys()

[]

In [53]:
df = transactions[transactions['block']<100000]

In [59]:
x = df.groupby('from').count()['to'].to_frame().reset_index().rename(columns={'from':'account','to':'cnt'})

In [64]:
x.cnt.value_counts()

2         2788
1         1414
3          495
4          290
5          213
6          151
7           84
8           69
9           37
10          29
11          28
13          18
14          18
12          15
18          13
15          10
16           9
23           9
17           8
21           7
26           7
20           6
38           6
22           5
24           4
25           4
35           3
29           3
32           3
43           3
          ... 
36           2
47           2
76           1
1068         1
976          1
156          1
148          1
48           1
28           1
40           1
3514         1
45           1
34           1
203          1
83           1
67           1
59           1
27           1
106          1
46           1
30           1
53           1
945          1
8893         1
427          1
117276       1
277          1
161          1
61           1
669          1
Name: cnt, Length: 64, dtype: int64

In [82]:
levels = pd.read_csv('genesisLevel.csv')

In [87]:
d_2 = levels[levels['level']==2]

In [88]:
d_2.head()

,address,level
13044,2910543af39aba0cd09dbb2d50200b3e800a63d2,2
13045,d9666150a9da92d9108198a4072970805a8b3428,2
13046,6896e22c387dc77eef9566aaecbf584a65130067,2
13047,0f5b063d39887707089ad6bee70c0fdd4de061ef,2
13048,b1abce2918e21ddb93aa452731a12672a3d9f75a,2


In [89]:
levels.count()

address    27670696
level      27670696
dtype: int64

In [91]:
len(levels['address'].unique())

27670696

In [92]:
transactions.head()

,from,to,value,time,block
649,6c26a66f05426f23e15b992c6ece6e01aa9c253a,6c26a66f05426f23e15b992c6ece6e01aa9c253a,1603530646850000000000,1439808196,100541
8189,9430a8b72fabc7a74bfb7ee077ad791ec06c13e8,4d4601e23b1dfe3cc1deae4d8dc080a3e74dc5de,3712000000000000000000,1455556418,1008849
41892,70089842ad51c663febe262e3fd47dab9ac7ff73,d04fd72ab9c4fe0f03ac001e20b5a740aa1b71c9,21877204000000001966080,1455421067,1000969
42457,70089842ad51c663febe262e3fd47dab9ac7ff73,d04fd72ab9c4fe0f03ac001e20b5a740aa1b71c9,1000000000000000,1455417870,1000787
46241,1a56a50c378d21d0aa544ed9a482300c7f6e78ec,119058dc2c577e9c4ba6914678aa9db565300ffe,323203436000000000000000,1439827863,101773


In [106]:
levels[levels['address']=='981c8bf3ecb76f43351f1213970592f6ba2d2dd8']

,address,level
27670692,981c8bf3ecb76f43351f1213970592f6ba2d2dd8,4998


In [105]:
all_transactions[all_transactions['to']=='dc753ce2cf9168499a06476e3fab40c60579dd87']

,from,to,value,time,block
538063,981c8bf3ecb76f43351f1213970592f6ba2d2dd8,dc753ce2cf9168499a06476e3fab40c60579dd87,5760650000000000000,1457200083,1104310


In [99]:
trans

,from,to,value,time,block


In [102]:
addresses

array(['034c2ebc51bb2ff05959035f8ac30318db337434',
       'd435d36d8f2130a55af8fcbe7c60b423117666d3'], dtype=object)

In [107]:
curr_level = 4999
level_dict = {}

In [108]:
while len(addresses)>0:
    curr_level+=1
    #if curr_level >= 5000:
    #    break
    print('looking into level: ',curr_level)
    for add in addresses:
        level_dict[add] = curr_level
    
    print('filled the dictionary level values')
    transactions = transactions[~transactions['to'].isin(addresses)]
    filtered_trans = transactions[transactions['from'].isin(addresses)]['to']
    addresses = filtered_trans.unique()
    #addresses = np.setdiff1d(new_addresses,addresses) # remove addresses of higher level
    print('obtained addresses for the next level')
    print('addresses obtained: ',len(addresses))

looking into level:  5000
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5001
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5002
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5003
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5004
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5005
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5006
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5007
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  500

obtained addresses for the next level
addresses obtained:  2
looking into level:  5071
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5072
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5073
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5074
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5075
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5076
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5077
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5078
filled the dictionary level values
obtained addresses for t

obtained addresses for the next level
addresses obtained:  2
looking into level:  5141
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5142
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5143
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5144
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5145
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5146
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5147
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5148
filled the dictionary level values
obtained addresses for t

obtained addresses for the next level
addresses obtained:  2
looking into level:  5211
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5212
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5213
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5214
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5215
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5216
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5217
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5218
filled the dictionary level values
obtained addresses for t

obtained addresses for the next level
addresses obtained:  2
looking into level:  5281
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5282
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5283
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5284
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5285
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5286
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5287
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  5288
filled the dictionary level values
obtained addresses for t

...

obtained addresses for the next level
addresses obtained:  1
looking into level:  9712
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9713
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9714
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9715
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9716
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9717
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9718
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9719
filled the dictionary level values
obtained addresses for t

obtained addresses for the next level
addresses obtained:  1
looking into level:  9785
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9786
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9787
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9788
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9789
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9790
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9791
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9792
filled the dictionary level values
obtained addresses for t

obtained addresses for the next level
addresses obtained:  1
looking into level:  9857
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9858
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9859
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9860
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9861
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9862
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9863
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9864
filled the dictionary level values
obtained addresses for t

obtained addresses for the next level
addresses obtained:  1
looking into level:  9929
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9930
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9931
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9932
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9933
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9934
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9935
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  9936
filled the dictionary level values
obtained addresses for t

obtained addresses for the next level
addresses obtained:  1
looking into level:  10001
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  10002
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1
looking into level:  10003
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  0
